# Notebook for running the snmachine pipeline on PLAsTiCC simulated data

This notebook illustrates the use of the `snmachine` supernova classification package by classifying a subset simulated data from the photometric light curve astronomical time-series classification challenge (PLAsTiCC). 

See Lochner et al. (2016): http://arxiv.org/abs/1603.00882

NOTE: The results here differ from the paper because we're using a much smaller dataset (in the interest of disk space and speed).

<img src="snmachine-plasticc.png" width=250>

This image illustrates the pipeline we use for this work. However, all the steps are modular and can be replaced. It is easy to write your own algorithm into the pipeline and change any of the stpes.

In [ ]:
import collections
import os
import pickle

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from snmachine import example_data, sndata, gps, snaugment, snfeatures, snclassifier, analysis, tsne_plot
from utils.plasticc_pipeline import create_folder_structure, get_directories, load_dataset

In [ ]:
%config Completer.use_jedi = False  # enable autocomplete

#### Aestetic settings

In [ ]:
%matplotlib nbagg

sns.set(font_scale=1.3, style="ticks")

In [ ]:
from __future__ import division
from snmachine import sndata, snfeatures, snclassifier, tsne_plot, example_data
import numpy as np
import matplotlib.pyplot as plt
import time, os, pywt,subprocess
from sklearn.decomposition import PCA
from astropy.table import Table,join,vstack
from astropy.io import fits
import sklearn.metrics 
import sncosmo

In [ ]:
%%capture --no-stdout 
#I use this to supress unnecessary warnings for clarity
%load_ext autoreload
%autoreload #Use this to reload modules if they are changed on disk while the notebook is running
%matplotlib nbagg

## Set up output structure

We make lots of output files so it makes sense to put them in one place. This is the recommended output file structure.

We start by **naming** our analysis.

In [ ]:
dataset_name = 'plasticc'
analysis_name = os.path.join(f'output_{dataset_name}', )

Then, we **write** in `folder_path` the path where to save the folder.

Note that `MultiNest` uses a hardcoded character limit for the output file names (we believe it is 100 characters). Thus, avoid making this file path to lengthy if using nested sampling otherwise the multinest output file names will be truncated.

In [ ]:
folder_path = '.'

Create the folder structure.

In [ ]:
create_folder_structure(folder_path, analysis_name)

See the folder structure. We save:
* the extracted features in `directories['features_directory']`
* the classification probabilities and ROC curves in `directories['classifications_directory']`
* the any intermediate files (such as multinest chains or GP fits) in `directories['intermediate_files_directory']`

In [ ]:
directories = get_directories(folder_path, analysis_name) 
directories

In [ ]:
path_saved_features = directories['features_directory']
path_saved_interm = directories['intermediate_files_directory']
path_saved_classifier = directories['classifications_directory']

## Initialise dataset object

Load the PLAsTiCC training data (hosted https://doi.org/10.5281/zenodo.2539456). For that, we **write** the path where the data is, and the name of the files.

In [ ]:
data_path = '../snmachine/example_data'

data_file_name = 'plasticc_train_lightcurves.csv'
metadata_file_name = 'plasticc_train_metadata.csv'

In [ ]:
# This cell takes ~2min to run
dataset = sndata.PlasticcData(folder=data_path, data_file=data_file_name,
                              metadata_file=metadata_file_name)

For now we restrict ourselves to three supernova types: Ia (90), II (42) and Ibc (62).

In [ ]:
metadata = dataset.metadata
is_snia = metadata.target == 90  # SN Ia
is_snibc = metadata.target == 62  # SN Ibc
is_snii = metadata.target == 42  # SN II

In [ ]:
objs_to_keep = metadata['object_id'][is_snia | is_snibc | is_snii]
objs_to_keep = np.array(objs_to_keep).flatten()

Update the dataset to only contain those SNe.

In [ ]:
dataset.update_dataset(objs_to_keep)

Now we can plot all the data and cycle through it (left and right arrows on your keyboard).

In [ ]:
dataset.plot_all()

Each light curve is represented in the Dataset object as an astropy table, compatible with `sncosmo`:

In [ ]:
dataset.data[dataset.object_names[0]]

## Preprocess dataset

**Write** the maximum duration of the gap to allowed in the light curves, `max_gap_length`.

In [ ]:
max_gap_length = 50

To remove all the gaps longer than `max_gap_length`, the `remove_gaps` function must be called a few times; it only removes the first gap longer than `max_gap_length`.

To introduce uniformity in the dataset, the resulting light curves are translated so their first observation is at time zero.

In [ ]:
dataset.remove_gaps(max_gap_length*2, verbose=True)
dataset.remove_gaps(max_gap_length*2, verbose=True)
dataset.remove_gaps(max_gap_length, verbose=True)
dataset.remove_gaps(max_gap_length, verbose=True)
dataset.remove_gaps(max_gap_length, verbose=True)

## Fit Gaussian Processes

Set the path to the folder to save the GP files.

In [ ]:
path_saved_gps = directories['intermediate_files_directory']

**Choose**:
- `t_min`: minimim time to evaluate the Gaussian Process Regression at.
- `t_max`: maximum time to evaluate the Gaussian Process Regression at.
- `gp_dim`: dimension of the Gaussian Process Regression. If  `gp_dim` is 1, the filters are fitted independently. If `gp_dim` is 2, the Matern kernel is used to fit light curves both in time and wavelength.
- `number_gp`: number of points to evaluate the Gaussian Process Regression at.
- `number_processes`: number of processors to use for parallelisation (**<font color=green>optional</font>**).

In [ ]:
t_min = 0
t_max = 278

gp_dim = 2
number_gp = 276
number_processes = 1

In [ ]:
gps.compute_gps(dataset, number_gp=number_gp, t_min=t_min, t_max=t_max, 
                gp_dim=gp_dim, output_root=path_saved_gps, 
                number_processes=number_processes)

## Augment Dataset

Our data augmentation requires previously fitted Gaussian Processes. So, if this was not done before, it needs to be done at this stage.

### Choose the Events to Augment<a name="chooseEvent"></a>

**Write** in `aug_obj_names` a list containing all the events to augment. Here we will try to augment them all.

In [ ]:
aug_obj_names = dataset.object_names  # try to augment all events

**Create** a dictionary that associates to each event, the target number of synthetic events to create from it. Note that some augmentations will fail so this is not the final number of events. Additionally, each class has a different creation efficiency.

In [ ]:
np.random.seed(42)
is_to_aug = np.in1d(dataset.object_names, aug_obj_names)

# Choose the target number of events in the augmented dataset. 
# Usually, only half of this number are accepted in the augmented dataset
target_number_aug = np.sum(is_to_aug) * 2

number_objs_per_label = collections.Counter(dataset.labels[is_to_aug])
number_aug_per_label = target_number_aug//len(number_objs_per_label.keys())
objs_number_to_aug = {}
for label in number_objs_per_label.keys():
    is_label = dataset.labels[is_to_aug] == label
    aug_is_label_obj_names = aug_obj_names[is_label]
    number_aug_per_obj = number_aug_per_label // np.sum(is_label)
    number_extra_aug_per_obj = number_aug_per_label % np.sum(is_label)
    extra_obj = np.random.choice(aug_is_label_obj_names, size=number_extra_aug_per_obj, 
                                 replace=False)
    objs_number_to_aug.update({obj: number_aug_per_obj for obj in aug_is_label_obj_names})
    objs_number_to_aug.update({obj: number_aug_per_obj+1 for obj in extra_obj})

In [ ]:
print(f'We aim to create up to {sum(objs_number_to_aug.values())} events.')  # confirm how many events to create

### Choose the Photometric Redshift<a name="choosePhotoZ"></a>

In order to simulate realistic photometric redshifts for the synthetic events, following [Boone (2019)](https://iopscience.iop.org/article/10.3847/1538-3881/ab5182) we chose a random event from the test set events that had a spectroscopic redshift measurement, and calculated the difference between its spectroscopic and photometric redshifts. We then added this difference to the true redshift of the augmented event to generate a photometric redshift. 

**Add** such a dataset containing spectroscopic and photometric redshift, and photometric redshift error of events as `z_table`. If none is provided, a similar table is generated from the events in `dataset`.

In [ ]:
# Here we use a small portion of the test data in interest of time
test_data_file_name = 'example_test_dataset.pckl'  
test_data_path = os.path.join(data_path, test_data_file_name)

test_data = load_dataset(test_data_path)
test_metadata = test_data.metadata

# Discard the events without spectroscopic redshift; 
# these are encoded with `hostgal_specz` equal to -9
z_table = test_metadata[test_metadata.hostgal_specz > -2]

### Run Augmentation<a name="aug"></a>

We also need to choose which survey to emulate in the augmentation. At the moment `snmachine` contains the Wide-Fast-Deep (WFD) and the Deep Drilling Field (DDF) survey of the Rubin Observatory Legacy Survey of Space and Time. Use `snaugment.PlasticcWFDAugment` for the former survey and `snaugment.PlasticcDDFAugment` for the latter.
You can also implement your own augmentation using those classes as an example.

In addition to the above inputs, we **chose** the random seed (`random_seed`) used to allow reproducible results and the maximum duration of the augmented light curves (`max_duration`).

The value of `max_duration` must be higher than the maximum duration of any light curve in `dataset`. If none is provided, `max_duration` is set to the length of the longest event in `dataset`.

In [ ]:
random_seed = 42 
max_duration = 278 

Here we augmented following the WFD survey. This step in the notebook takes ~25min.

In [ ]:
aug = snaugment.PlasticcWFDAugment(dataset=dataset, path_saved_gps=path_saved_gps, 
                                   objs_number_to_aug=objs_number_to_aug,
                                   random_seed=random_seed, max_duration=max_duration, 
                                   z_table=z_table)

In [ ]:
aug.augment()

### Balance augmented set

We now create a balanced augmented training set with the same number of events from each class.

In [ ]:
only_new_train_data = aug.only_new_dataset
only_new_meta = only_new_train_data.metadata

In [ ]:
is_42 = only_new_meta.target == 42
is_62 = only_new_meta.target == 62
is_90 = only_new_meta.target == 90

In [ ]:
size_balance = min(np.sum(is_42), np.sum(is_62), np.sum(is_90))
size_balance

In [ ]:
np.random.seed(42)
objs_to_keep = []

is_sniabcii = [is_42, is_62, is_90]
for i, is_sn in enumerate(is_sniabcii):
    indexes = np.where(is_sn)[0]
    try:
        indexes_to_stay = np.random.choice(
            indexes, size=size_balance, replace=False)
    except ValueError:
        print(f'The class {only_new_meta.target[is_sn][0]} only has {len(indexes)} events.')
        indexes_to_stay = indexes
    objs_to_stay = is_sn[indexes_to_stay].index.to_numpy()
    objs_to_keep.append(objs_to_stay)
objs_to_keep = np.concatenate(objs_to_keep)

In [ ]:
new_metadata = only_new_meta.loc[objs_to_keep]
only_new_train_data.object_names = list(new_metadata.index)
only_new_train_data.update_dataset(list(new_metadata.index))
only_new_train_data.update_dataset(list(only_new_train_data.metadata.index))

In [ ]:
# number of events per SNe class
collections.Counter(only_new_train_data.metadata['target'])  

Save the new augmented dataset.

In [ ]:
augmented_data_dir = folder_path
aug_file_name = 'aug_plasticc.pckl'

In [ ]:
with open(os.path.join(augmented_data_dir, aug_file_name), 'wb') as f:
    pickle.dump(only_new_train_data, f, pickle.HIGHEST_PROTOCOL)

## Extract features for the data

The next step is to extract useful features from the data. This can often take a long time, depending on the feature extraction method, so it's a good idea to save these to file.

We create a new folder to hold the results relative to the augmented training set.

In [ ]:
analysis_name = aug_file_name[:-5]
create_folder_structure(folder_path, analysis_name)

In [ ]:
# We redefine these variables because we will not need the older definitions
directories = get_directories(folder_path, analysis_name) 
path_saved_gps = directories['intermediate_files_directory']

### Run Gaussian Processes.

In [ ]:
gps.compute_gps(only_new_train_data, number_gp=number_gp, 
                t_min=t_min, t_max=t_max, 
                gp_dim=gp_dim, output_root=path_saved_gps, 
                number_processes=number_processes)

### Wavelet Decomposition<a name="waveletDecomp"></a>

Now, we do a wavelet decomposition of the events. **Write** in `path_saved_wavelets` the path to the folder where to save them.

In [ ]:
path_saved_wavelets = directories['intermediate_files_directory']

Following [Lochner et al. (2016)](https://iopscience.iop.org/article/10.3847/0067-0049/225/2/31), we then reduced the dimensionality of this wavelet space using Principal Component Analysis (PCA). Therefore, **choose** the number of PCA components to keep (`number_comps`) and **write** the path to the folder where to save the reduced wavelets (`path_saved_reduced_wavelets`).

In [ ]:
number_comps = 40
path_saved_reduced_wavelets = directories['features_directory']

Perform the wavelet decomposition and dimensionality reduction.

In [ ]:
wf = snfeatures.WaveletFeatures(output_root=path_saved_wavelets)

reduced_wavelet_features = wf.compute_reduced_features(
    only_new_train_data, number_comps=number_comps, 
    **{'wavelet_name': 'sym2', 'number_decomp_levels': 2,
       'path_save_eigendecomp': path_saved_reduced_wavelets})

Save the reduced features.

In [ ]:
wf.save_reduced_features(reduced_wavelet_features, path_saved_reduced_wavelets)

Include Redshift Information<a name="addZ"></a>

In [paper]() we found that photometric redshift and its uncertainty are crucial for classification. Therefore, in the cell bellow, we include these properties as features. **Modify** it to include other properties as features.

In [ ]:
features = reduced_wavelet_features.copy()  # only the wavelet features

metadata = only_new_train_data.metadata
features['hostgal_photoz'] = metadata.hostgal_photoz.values.astype(float)
features['hostgal_photoz_err'] = metadata.hostgal_photoz_err.values.astype(float)

To save the features, **write** in `saved_features_path` the path to the folder where to save them.

In [ ]:
path_saved_features = directories['features_directory']

In [ ]:
features.to_pickle(os.path.join(path_saved_features, 'features.pckl'))

data_labels = only_new_train_data.labels.astype(int)  # class label of each event
data_labels.to_pickle(os.path.join(path_saved_features, 'data_labels.pckl'))

We also make a t-SNE plot.

In [ ]:
#plt.figure(figsize=(2,3))
#tsne_plot.plot(features, data_labels) # Not yet implemented for PLAsTiCC data

## Classify

Finally, we're ready to run the machine learning algorithm. There's a utility function in the `snclassifier` library to make it easy to run all the algorithms available, including converting features to `numpy` arrays and rescaling them and automatically generating ROC curves and metrics. Hyperparameters are automatically selected using a grid search combined with cross-validation. All functionality can also be individually run from `snclassifier`.

Here we chose a LightGBM classifier.

In [ ]:
classifier_instance = snclassifier.LightGBMClassifier(classifier_name='our_classifier', random_seed=42)

We can now train and use the classifier generated above or optimise it beforehand. In general, it is important to optimise the classifier hyperparameters.

In the paper we use the default optimisation but in interest of time, we do here a shorter optimisation.

In [ ]:
param_grid={'learning_rate': [.1, .25, .5]}

classifier_instance.optimise(features, data_labels, param_grid=param_grid, 
                             scoring='logloss', 
                             number_cv_folds=5, metadata=metadata)

### Test set

We use a small set of the test set here. 

Before classifying the test set events, we need to obtain their features. 

#### Preprocess light curves

In [ ]:
test_data.remove_gaps(max_gap_length*2, verbose=True)
test_data.remove_gaps(max_gap_length*2, verbose=True)
test_data.remove_gaps(max_gap_length, verbose=True)
test_data.remove_gaps(max_gap_length, verbose=True)
test_data.remove_gaps(max_gap_length, verbose=True)

#### Extract Features

Fit Gaussian Processes

In [ ]:
test_analysis_name = test_data_file_name[:-5]
create_folder_structure(folder_path, test_analysis_name)

In [ ]:
test_directories = get_directories(folder_path, test_analysis_name) 
test_path_saved_gps = test_directories['intermediate_files_directory']

In [ ]:
gps.compute_gps(test_data, number_gp=number_gp, t_min=t_min, t_max=t_max, 
                gp_dim=gp_dim, output_root=test_path_saved_gps, 
                number_processes=number_processes)

Now, we do a wavelet decomposition of the test set events and project them into the feature space obtained with the training set. 

In [ ]:
test_path_saved_wavelets = directories['intermediate_files_directory']

In [ ]:
wf = snfeatures.WaveletFeatures(output_root=test_path_saved_wavelets)

test_reduced_wavelet_features = wf.compute_reduced_features(
    test_data, number_comps=number_comps, 
    path_saved_eigendecomp=path_saved_reduced_wavelets,
    **{'wavelet_name': 'sym2', 'number_decomp_levels': 2})

Include Redshift Information

In [ ]:
test_features = test_reduced_wavelet_features.copy()  # only the wavelet features

test_metadata = test_data.metadata
test_features['hostgal_photoz'] = test_metadata.hostgal_photoz.values.astype(float)
test_features['hostgal_photoz_err'] = test_metadata.hostgal_photoz_err.values.astype(float)

#### Classify Test Set

Compute the predicted class (`y_pred`) and the probability of belonging to each different class (`y_probs`). Note that the predicted class is the one with the highest probability.

In [ ]:
classifier = classifier_instance.classifier
y_pred_test = classifier.predict(test_features)
y_probs_test = classifier.predict_proba(test_features)

#### Performance

If we know the true class label of each event we can calculate the performance of the classifier. Otherwise, our predictions are saved in `y_pred_test` and `y_probs_test`.

In this example we know the true class labels.

In [ ]:
test_data_labels = test_metadata.true_target  # class label of each event

We start by computing the Area under the ROC Curve (AUC) and the PLAsTiCC logloss. For that, choose which class to consider as *positive* (the other classes will be considered *negative*). Then, **write** in `which_column` the column that corresponds to that class. Note that the class order is accessed through the classifier.

In [ ]:
which_column = 2  # we are interested in SN Ia vs others

In [ ]:
classifier.which_column = which_column
auc_test = snclassifier.auc_score(classifier=classifier, X_features=test_features, 
                                  y_true=test_data_labels, which_column=which_column)
logloss_test = snclassifier.logloss_score(classifier=classifier, X_features=test_features, 
                                          y_true=test_data_labels)
print('{:^10} {:^10} {:^10}'.format('', 'AUC', 'Logloss'))
print('{:^10} {:^10.3f} {:^10.3f}'.format('test', auc_test, logloss_test))

Now, we plot the confusion matrix.

In [ ]:
title = 'Confusion matrix'
analysis.plot_confusion_matrix(test_data_labels, y_pred_test, normalise='accuracy', title=title,
                               dict_label_to_real=analysis.dict_label_to_real_plasticc)

ROC Curves

In [ ]:
analysis.plot_classifier_roc_curve(test_data_labels, y_probs_test,
                                   dict_label_to_real=analysis.dict_label_to_real_plasticc)

[Go back to top.](#index)